# More SQL Tools

In [1]:
import pandas as pd
import sqlite3
import pandasql

![sql](sql-logo.jpg)

## Agenda

SWBAT:

- Use `GROUP BY` for SQL queries of aggregates;
- Use `CASE WHEN` as an "if" in more complex SQL queries;
- Construct SQL queries that involve subqueries;
- Connect SQL with `pandas`;
- Describe various dialects of SQL;
- Describe PostgreSQL as a more sophisticated alternative to SQLite;

In [2]:
cur = sqlite3.connect('flights.db').cursor()

## Grouping statements

Combine `SELECT` and `GROUP BY` when you want *aggregates* by values

`SELECT` `COUNT`, `MIN(x)`, `MAX(x)`, `SUM(x)`, etc.

`GROUP BY x`

**Task**<br>
- Which countries have the highest numbers of active airlines? Return the top 25.

In [3]:
cur.execute(
"""
SELECT COUNT() num, country
FROM airlines
WHERE active='Y'
GROUP BY country
ORDER BY num DESC
LIMIT 25
"""
).fetchall()

[(141, 'United States'),
 (72, 'Russia'),
 (40, 'United Kingdom'),
 (37, 'Germany'),
 (34, 'Canada'),
 (26, 'Australia'),
 (25, 'China'),
 (24, 'Spain'),
 (23, 'Brazil'),
 (22, 'France'),
 (19, 'Japan'),
 (18, 'Italy'),
 (17, 'Indonesia'),
 (17, 'India'),
 (16, 'Turkey'),
 (16, 'Thailand'),
 (15, 'Sweden'),
 (14, 'Switzerland'),
 (13, 'Ukraine'),
 (13, 'Portugal'),
 (12, 'Mexico'),
 (12, 'Finland'),
 (12, 'Egypt'),
 (12, 'Austria'),
 (11, 'Peru')]

**Exercise**:
- Which countries have the highest numbers of inactive airlines? Return all the countries that have more than 10.

In [13]:
# Your code here
cur.execute(
"""
SELECT count() num, country 
FROM airlines
WHERE (active='N') 
GROUP BY country
HAVING num>10
ORDER BY num DESC
"""
).fetchall()

[(939, 'United States'),
 (427, 'Mexico'),
 (367, 'United Kingdom'),
 (284, 'Canada'),
 (158, 'Russia'),
 (142, 'Spain'),
 (97, 'France'),
 (94, 'Germany'),
 (81, 'South Africa'),
 (80, 'Nigeria'),
 (76, 'Ukraine'),
 (73, 'Kazakhstan'),
 (72, 'Italy'),
 (67, 'Australia'),
 (55, 'Sweden'),
 (47, 'Sudan'),
 (46, 'Switzerland'),
 (45, 'China'),
 (43, 'Netherlands'),
 (38, 'Austria'),
 (36, 'Egypt'),
 (35, 'Chile'),
 (35, 'Brazil'),
 (35, 'Angola'),
 (34, 'Colombia'),
 (32, 'Thailand'),
 (32, 'Portugal'),
 (32, 'Mauritania'),
 (31, 'Indonesia'),
 (29, 'Kyrgyzstan'),
 (27, 'Czech Republic'),
 (25, 'Japan'),
 (25, 'Dominican Republic'),
 (25, 'Denmark'),
 (25, 'Bulgaria'),
 (24, 'New Zealand'),
 (24, 'Iran'),
 (23, 'United Arab Emirates'),
 (23, 'Turkey'),
 (23, 'Sierra Leone'),
 (23, 'Norway'),
 (23, 'Libya'),
 (22, 'Venezuela'),
 (22, 'Pakistan'),
 (21, 'Uganda'),
 (21, 'Ireland'),
 (21, 'Belgium'),
 (20, 'Zambia'),
 (20, 'Kenya'),
 (19, 'Greece'),
 (19, 'Ecuador'),
 (19, 'Democratic Repub

**Exercise**:

- Run a query that will return the number of airports by time zone. Each row should have a number of airports and a time zone.

In [19]:
# Your code here
pd.DataFrame(cur.execute(
"""
SELECT timezone, count() airports 
FROM airports
GROUP BY timezone

"""
).fetchall())

,0,1
0,Africa/Abidjan,7
1,Africa/Accra,6
2,Africa/Addis_Ababa,31
3,Africa/Algiers,44
4,Africa/Asmera,4
...,...,...
289,Pacific/Tongatapu,6
290,Pacific/Truk,3
291,Pacific/Wake,1
292,Pacific/Wallis,2


### `CASE WHEN`

`CASE WHEN` is SQL's version of `if ... then ... else`. It must *always* be closed with an `END`.

In [ ]:
# What will this query return?

cur.execute(
"""
SELECT name, city,
CASE WHEN latitude > 0 THEN 'northern hemisphere'
ELSE 'southern hemisphere'
END AS hemisphere
FROM airports
"""
).fetchall()

### Subqueries

For more complex queries it can be helpful to break them down into multiple parts. Subqueries are a natural way to do this.

Suppose I wanted to know, after collecting together the highest airport in each country, which one's name comes alphabetically first.

I might break this down by first collecting the highest airports and then _wrapping_ that query in a higher query that selects the name and country I want _from_ the result of that first query:

In [ ]:
cur.execute(
"""
SELECT MIN(name), country, altitude FROM
(SELECT name, code, country, MAX(CAST(altitude AS INT)) AS altitude
FROM airports
GROUP BY country)
"""
).fetchall()

## Using sql within `pandas` to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

### Get data for an example

In [ ]:
shelter_data=pd.read_csv('https://data.austintexas.gov/api/\
views/9t4d-g238/rows.csv?accessType=DOWNLOAD')

In [ ]:
shelter_data[shelter_data['Name'] == "Oscar"].head()

### With .query()

In [ ]:
oscar_data = shelter_data.query('Name == "Oscar"')
oscar_data.head()

## Transfering from sqlite to pandas

`.read_sql()`

In [ ]:
conn = sqlite3.connect("flights.db")
df = pd.read_sql("SELECT * FROM airports LIMIT 50", conn)
df

**Exercise for later**: 
Convert one of the earlier queries in the lesson to a pandas data frame



`pandasql` is a library that allows SQL-style querying of `pandas` DataFrames.

In [ ]:
pandasql.sqldf("SELECT name, code FROM df LIMIT 10", env=globals())

## Other Dialects of SQL

There are many versions of SQL out there! Here are a few of the major players in the workplace:

- SQLite (we've already seen this!)
- T-SQL (Transact-SQL, used by Microsoft)
- PostgreSQL (free and open-source!)
- Oracle SQL
- MySQL (half open-souce, half Oracle)

### More Resources

- [blog](https://learnsql.com/blog/what-sql-dialect-to-learn/) <br/>
- There's a whole [wikibook](https://en.wikibooks.org/wiki/SQL_Dialects_Reference) on this!

## Demo of SQL Interface on data.world

## Demo of PostgreSQL